<a href="https://colab.research.google.com/github/bartteeuwen/USD-Applied-Data-Mining/blob/main/Predicting_Responder_Outcomes_In_A_Lung_Cancer_Drug_Trial_Using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(conflicted)
conflict_prefer("SMOTE", "smotefamily")

[conflicted] Will prefer smotefamily::SMOTE over any other package.


In [5]:
# Create a list of the packages you need
my_packages <- c(
  "readr", "dplyr", "ggplot2", "tidyr", "GGally",
  "corrplot", "corrr", "ggcorrplot", "naniar", "car",
  "tidyverse", "janitor", "recipes", "smotefamily",
  "caret", "doParallel", "scales"
)

# Install the packages
install.packages(my_packages)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘listenv’, ‘parallelly’, ‘quadprog’, ‘quantmod’, ‘future’, ‘globals’, ‘fracdiff’, ‘lmtest’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘shape’, ‘future.apply’, ‘progressr’, ‘SQUAREM’, ‘permute’, ‘cowplot’, ‘Deriv’, ‘forecast’, ‘microbenchmark’, ‘rbibutils’, ‘diagram’, ‘lava’, ‘patchwork’, ‘ca’, ‘colorspace’, ‘gclus’, ‘qap’, ‘registry’, ‘TSP’, ‘vegan’, ‘gridExtra’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘Rdpack’, ‘RcppEigen’, ‘prodlim’, ‘proxy’, ‘ggstats’, ‘ggrepel’, ‘seriation’, ‘reshape2’, ‘norm’, ‘visdat’, ‘viridis’, ‘UpSetR’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘snakecase’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘timeDate’, ‘FNN’, ‘dbscan’, ‘igraph’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘iterators’




In [4]:
library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(GGally)
library(corrplot)
library(corrr)
library(ggcorrplot)
library(naniar)
library(car)
library(tidyverse)
library(janitor)
library(recipes)
library(smotefamily)
library(caret)
library(doParallel)
library(scales)

ERROR: Error in library(package, pos = pos, lib.loc = lib.loc, character.only = TRUE, : there is no package called ‘corrplot’


In [ ]:
df <- read_csv("synthetic_moderna_mrna4157_trial (1).csv")

In [ ]:
# Ensure target is a factor
df <- df %>%
mutate(treatment_outcome = factor(treatment_outcome,
levels = c("NonResponder", "Responder")))
head(df)

In [ ]:
colnames(df)

In [ ]:
str(df)

In [ ]:
# Barplot
ggplot(df, aes(x = trial_arm, fill = treatment_outcome)) +
geom_bar(position = "fill") +
scale_y_continuous(labels = scales::percent_format()) +
labs(
title = "Treatment Outcome Distribution by Trial Arm",
y = "Proportion", x = "Trial Arm"
) +
theme_minimal()

In [ ]:
ggplot(df, aes(x = trial_arm, fill = side_effect)) +
geom_bar(position = "fill") +
scale_y_continuous(labels = percent_format()) +
labs(
title = "Side Effect Distribution by Trial Arm",
y = "Proportion",
x = "Trial Arm",
fill = "Side Effect"
) +
theme_minimal()

In [ ]:
# Subset df to numeric columns for correlation
num_data <- df %>%
select(where(is.numeric))
cor_matrix <- cor(num_data, use = "pairwise.complete.obs")
ggcorrplot(cor_matrix,
type = "upper",
lab = TRUE,
lab_size = 3.5,
colors = c("red", "white", "blue"),
title = "Correlation Matrix of Numeric Variables",
ggtheme = theme_minimal())

In [ ]:
# Pairwise plots
ggpairs(df,
columns = c("age", "immune_response_score", "biomarker_A", "biomarker_B", "tumor_volume_change"mapping = aes(color = treatment_outcome),
upper = list(continuous = wrap("cor", size = 3)),
lower = list(continuous = wrap("points", alpha = 0.4, size = 0.7)),
diag = list(continuous = wrap("densityDiag", alpha = 0.5))) +
theme_minimal()

In [ ]:
# Summary
table(df$trial_arm, df$treatment_outcome)

In [ ]:
# Violin plots by treatment outcome
df %>%
  pivot_longer(
    cols = c("age", "immune_response_score", "biomarker_A", "biomarker_B", "tumor_volume_change"),
    names_to = "feature",
    values_to = "value"
  ) %>%
  ggplot(aes(x = treatment_outcome, y = value, fill = treatment_outcome)) +
  geom_violin(trim = FALSE, alpha = 0.6) +
  facet_wrap(~ feature, scales = "free_y") +
  theme_minimal() +
  labs(
    title = "Distribution of Numeric Features by Treatment Outcome",
    y = NULL,
    x = "Treatment Outcome"
  )

In [ ]:
# Histograms numeric
df %>%
select(where(is.numeric), -participant_id) %>%
pivot_longer(cols = everything(), names_to = "feature", values_to = "value") %>%
ggplot(aes(x = value, fill = feature)) +
geom_histogram(bins = 30, alpha = 0.7, color = "white") +
facet_wrap(~ feature, scales = "free") +
theme_minimal() +
labs(title = "Histograms of Numeric Predictors", x = NULL, y = "Count")

In [ ]:
# Boxplots by trial\_arm
df %>%
pivot_longer(cols = c("age", "immune_response_score", "biomarker_A", "biomarker_B", "tumor_volume_channames_to = "feature", values_to = "value") %>%
ggplot(aes(x = trial_arm, y = value, fill = trial_arm)) +
geom_boxplot(outlier.alpha = 0.4) +
facet_wrap(~ feature, scales = "free", ncol = 3) +
theme_minimal() +
labs(title = "Boxplots of Numeric Features by Trial Arm", x = "Trial Arm", y = NULL)

In [ ]:
# Missingness overview
naniar::miss_var_summary(df)

In [ ]:
# Count outliers per variable
count_outliers <- function(x) {
qnt <- quantile(x, probs = c(0.25, 0.75), na.rm = TRUE)
iqr <- qnt[2] - qnt[1]
lower <- qnt[1] - 1.5 * iqr
upper <- qnt[2] + 1.5 * iqr
sum(x < lower | x > upper, na.rm = TRUE)
}

df %>%
select(where(is.numeric)) %>%
summarise(across(everything(), count_outliers)) %>%
pivot_longer(everything(), names_to = "feature", values_to = "outlier_count") %>%
arrange(desc(outlier_count)) %>%
print()

In [ ]:
df %>%
pivot_longer(cols = c("age", "immune_response_score", "biomarker_A", "biomarker_B", "tumor_volume_channames_to = "feature", values_to = "value") %>%
ggplot(aes(x = "", y = value)) +
geom_boxplot(outlier.color = "red", outlier.shape = 16, outlier.size = 2) +
facet_wrap(~ feature, scales = "free_y") +
labs(title = "Outlier Detection using Boxplots", x = NULL, y = NULL) +
theme_minimal()

**Pre-Processing & Cleaning**

In [ ]:
# Ensure categorical types
df <- df %>%
mutate(
treatment_outcome = factor(treatment_outcome, levels = c("NonResponder", "Responder")),
trial_arm = factor(trial_arm),
side_effect = factor(side_effect)
) %>%
clean_names()

# Recipe for imputation and scaling
rec <- recipe(treatment_outcome ~ ., data = df) %>%
step_rm(participant_id) %>%
step_impute_median(all_numeric_predictors()) %>%
step_impute_mode(all_nominal_predictors()) %>%
step_dummy(all_nominal_predictors(), one_hot = TRUE) %>%
step_center(all_numeric_predictors()) %>%
step_scale(all_numeric_predictors())

# Prep and bake for clean, model-ready 'df_clean'
df_clean <- prep(rec) %>% bake(new_data = NULL)
glimpse(df_clean)

In [ ]:
# Outlier handling
rec <- rec %>%
step_mutate_at(all_numeric_predictors(), fn = ~ifelse(. > quantile(., 0.99), quantile(., 0.99),
ifelse(. < quantile(., 0.01), quantile(., 0.01)

# Stratified Train-Test Split
set.seed(123)
split_idx <- createDataPartition(df_clean$treatment_outcome, p = 0.7, list = FALSE)
train_clean <- df_clean[split_idx, ]
test_clean <- df_clean[-split_idx, ]

# SMOTE rebalance
X_train <- train_clean %>% select(-treatment_outcome)
y_train <- train_clean$treatment_outcome
sm <- SMOTE(X_train, target = y_train, K = 5)

In [ ]:
df_clean_bal <- bind_cols(sm$data %>% select(-class),
treatment_outcome = factor(sm$data$class, levels = levels(y_train)))
glimpse(df_clean_bal)

In [ ]:
colnames(df_clean)

**Test Pre-processing Success**

In [ ]:
library(broom)
library(knitr)
library(kableExtra)
library(dplyr)
library(purrr)

# Correct numeric vars post-cleaning
num_vars <- c("age", "immune_response_score", "biomarker_a", "biomarker_b", "tumor_volume_change")
# ANOVA
anova_tbl <- purrr::map_dfr(num_vars, function(v) {
model <- aov(formula = as.formula(paste(v, "~ treatment_outcome")), data = df_clean)
tidy(model) %>%
dplyr::filter(term != "Residuals") %>%
mutate(variable = v, test = "ANOVA") %>%
select(variable, test, statistic, p_value = p.value)
})
# Kruskal-Wallis
kw_tbl <- purrr::map_dfr(num_vars, function(v) {
kw <- kruskal.test(formula = as.formula(paste(v, "~ treatment_outcome")), data = df_clean)
tibble(
variable = v,
test = "Kruskal-Wallis",
statistic = kw$statistic,
p_value = kw$p.value
)
})
# Combine and show results
all_tests_tbl <- bind_rows(anova_tbl, kw_tbl) %>%
mutate(across(where(is.numeric), ~ round(., 3))) %>%
arrange(p_value)
kable(all_tests_tbl,
caption = "Hypothesis Tests: ANOVA, Kruskal-Wallis (post-cleaning, df\\_clean)",
align = "lccc") %>%
kable_styling(full_width = FALSE, bootstrap_options = c("striped", "hover"))

**VIF**

In [ ]:
library(car)
library(dplyr)

# Select numeric predictors only (exclude outcome)
num_vars <- df_clean %>%
select(age, immune_response_score, biomarker_a, biomarker_b, tumor_volume_change)

# Add treatment outcome temporarily for formula
vif_df <- df_clean %>%
select(treatment_outcome, age, immune_response_score, biomarker_a, biomarker_b, tumor_volume_change) %mutate(treatment_outcome = as.factor(treatment_outcome))

# Fit a linear model with all predictors
vif_model <- lm(as.numeric(treatment_outcome) ~ ., data = vif_df)

# Calculate VIF
vif_values <- vif(vif_model) %>% as.data.frame() %>%
tibble::rownames_to_column("Variable") %>%
rename(VIF = ".")
print(vif_values)

**PCA Analysis**

In [ ]:
library(ggplot2)
library(factoextra)

# Prepare numeric predictors
pca_data <- df_clean %>%
select(age, immune_response_score, biomarker_a, biomarker_b, tumor_volume_change) %>%
mutate(across(everything(), ~ ifelse(is.finite(.), ., NA_real_))) %>%
mutate(across(everything(), ~ ifelse(is.na(.), median(., na.rm = TRUE), .)))

# Run PCA
pca_result <- prcomp(pca_data, center = TRUE, scale. = TRUE)

# Scree plot (explained variance)
fviz_eig(pca_result, addlabels = TRUE, barfill = "steelblue") +
labs(title = "PCA Scree Plot (Explained Variance)")

In [ ]:
# PCA biplot colored by treatment outcome
fviz_pca_biplot(
pca_result,
geom.ind = "point",
habillage = df$treatment_outcome,
addEllipses = TRUE,
repel = TRUE,
label = "var",
title = "PCA Biplot: Numeric Predictors colored by Treatment Outcome"
)

**Modeling and Hyperparameter Tuning Pipeline**

In [ ]:
library(tidymodels)
library(caret)
library(glmnet)
library(kernlab)
library(xgboost)
library(doParallel)

df <- read_csv("moderna_cancer_trial (1).csv")

In [ ]:
df_model <- df %>%
select(
age,
immune_response_score,
biomarker_B,
tumor_volume_change,
trial_arm,
side_effect,
treatment_outcome
) %>%
mutate(
treatment_outcome = factor(treatment_outcome, levels = c("NonResponder", "Responder")),
trial_arm = factor(trial_arm),
side_effect = factor(side_effect)
)

# Split
set.seed(123)
split <- initial_split(df_model, strata = treatment_outcome)
train <- training(split)
test <- testing(split)

# Preprocessing
pp <- recipe(treatment_outcome ~ ., data = train) %>%
step_impute_median(all_numeric_predictors()) %>%
step_impute_mode(all_nominal_predictors()) %>%
step_dummy(all_nominal_predictors()) %>%
step_center(all_numeric_predictors()) %>%
step_scale(all_numeric_predictors()) %>%
prep()
df_clean_bal <- bake(pp, new_data = NULL)

# Prepare X and y
X <- df_clean_bal %>% select(-treatment_outcome)
y <- df_clean_bal$treatment_outcome

# Setup parallel processing
cl <- makePSOCKcluster(parallel::detectCores() - 1)
registerDoParallel(cl)

# CV control
ctrl <- trainControl(
method = "cv",
number = 3,
classProbs = TRUE,
summaryFunction = twoClassSummary,
savePredictions = "final",
verboseIter = FALSE,
allowParallel = TRUE
)

# Hyperparameter grids
grid_glmnet <- expand.grid(
alpha = c(0, 0.5, 1),
lambda = 10ˆseq(-3, -1, length.out = 3)
)
grid_rf <- expand.grid(
  mtry = unique(pmax(1, round(c(sqrt(ncol(X))/2, sqrt(ncol(X)), sqrt(ncol(X))*1.5))))
)
sig0 <- kernlab::sigest(as.matrix(X))[2]
grid_svm <- expand.grid(
sigma = sig0,
C = c(0.1, 1, 10)
)
grid_xgb <- expand.grid(
nrounds = 50,
max_depth = c(3, 5),
eta = c(0.05, 0.1),
gamma = 0,
colsample_bytree = 0.8,
min_child_weight = 1,
subsample = 0.8
)
# Fit models
set.seed(123)
mod_glmnet <- train(
x = X, y = y,
method = "glmnet",
metric = "ROC",
tuneGrid = grid_glmnet,
trControl = ctrl
)

In [ ]:
mod_rf <- train(
x = X, y = y,
method = "rf",
metric = "ROC",
tuneGrid = grid_rf,
trControl = ctrl
)

In [ ]:
mod_svm <- train(
x = X, y = y,
method = "svmRadial",
metric = "ROC",
tuneGrid = grid_svm,
trControl = ctrl
)

In [ ]:
mod_xgb <- train(
x = X, y = y,
method = "xgbTree",
metric = "ROC",
tuneGrid = grid_xgb,
trControl = ctrl
)

In [ ]:
# Stop cluster
stopCluster(cl)
registerDoSEQ()

# Review results
print(mod_glmnet)

In [ ]:
print(mod_rf)

In [ ]:
print(mod_svm)

In [ ]:
print(mod_xgb)

In [ ]:
# Plot hyperparameter tuning visually
plot(mod_glmnet, main = "GLMnet Hyperparameter Tuning")

In [ ]:
plot(mod_rf, main = "Random Forest Hyperparameter Tuning")

In [ ]:
plot(mod_svm, main = "SVM Radial Hyperparameter Tuning")

In [ ]:
plot(mod_xgb, main = "XGBoost Hyperparameter Tuning")

**Model evaluation**

In [ ]:
library(tidymodels)
library(pROC)
library(caret)
library(ggplot2)

# Prepare test data
test_clean <- bake(pp, new_data = test)
X_test <- test_clean %>% select(-treatment_outcome)
y_test <- test_clean$treatment_outcome

# Evaluate GLMnet
glmnet_probs <- predict(mod_glmnet, X_test, type = "prob")[, "Responder"]
glmnet_preds <- predict(mod_glmnet, X_test)
roc_glmnet <- roc(response = y_test, predictor = glmnet_probs, levels = c("NonResponder", "Responder"))

In [ ]:
auc_glmnet <- auc(roc_glmnet)
print(paste("GLMnet AUC:", round(auc_glmnet, 3)))

In [ ]:
plot(roc_glmnet, col = "purple", lwd = 2, main = "ROC Curve - GLMnet")
abline(a = 0, b = 1, lty = 2, col = "gray")

In [ ]:
cm_glmnet <- confusionMatrix(glmnet_preds, y_test, positive = "Responder")
print(cm_glmnet)

In [ ]:
print(paste("GLMnet Kappa:", round(cm_glmnet$overall["Kappa"], 3)))

In [ ]:
# Evaluate Random Forest
rf_probs <- predict(mod_rf, X_test, type = "prob")[, "Responder"]
rf_preds <- predict(mod_rf, X_test)
roc_rf <- roc(response = y_test, predictor = rf_probs, levels = c("NonResponder", "Responder"))

In [ ]:
auc_rf <- auc(roc_rf)
print(paste("Random Forest AUC:", round(auc_rf, 3)))

In [ ]:
plot(roc_rf, col = "steelblue", lwd = 2, main = "ROC Curve - Random Forest")
abline(a = 0, b = 1, lty = 2, col = "gray")

In [ ]:
cm_rf <- confusionMatrix(rf_preds, y_test, positive = "Responder")
print(cm_rf)

In [ ]:
print(paste("Random Forest Kappa:", round(cm_rf$overall["Kappa"], 3)))

In [ ]:
# Evaluate SVM
svm_probs <- predict(mod_svm, X_test, type = "prob")[, "Responder"]
svm_preds <- predict(mod_svm, X_test)
roc_svm <- roc(response = y_test, predictor = svm_probs, levels = c("NonResponder", "Responder"))

In [ ]:
auc_svm <- auc(roc_svm)
print(paste("SVM AUC:", round(auc_svm, 3)))

In [ ]:
plot(roc_svm, col = "darkgreen", lwd = 2, main = "ROC Curve - SVM")
abline(a = 0, b = 1, lty = 2, col = "gray")

In [ ]:
cm_svm <- confusionMatrix(svm_preds, y_test, positive = "Responder")
print(cm_svm)

In [ ]:
print(paste("SVM Kappa:", round(cm_svm$overall["Kappa"], 3)))

In [ ]:
# Evaluate XGBoost
xgb_probs <- predict(mod_xgb, X_test, type = "prob")[, "Responder"]
xgb_preds <- predict(mod_xgb, X_test)
roc_xgb <- roc(response = y_test, predictor = xgb_probs, levels = c("NonResponder", "Responder"))

In [ ]:
auc_xgb <- auc(roc_xgb)
print(paste("XGBoost AUC:", round(auc_xgb, 3)))

In [ ]:
plot(roc_xgb, col = "firebrick", lwd = 2, main = "ROC Curve - XGBoost")
abline(a = 0, b = 1, lty = 2, col = "gray")

In [ ]:
cm_xgb <- confusionMatrix(xgb_preds, y_test, positive = "Responder")
print(cm_xgb)

In [ ]:
print(paste("XGBoost Kappa:", round(cm_xgb$overall["Kappa"], 3)))

In [ ]:
# Combined Model Comparison Table
results <- data.frame(
Model = c("GLMnet", "Random Forest", "SVM", "XGBoost"),
AUC = c(auc_glmnet, auc_rf, auc_svm, auc_xgb),
Kappa = c(cm_glmnet$overall["Kappa"], cm_rf$overall["Kappa"],
cm_svm$overall["Kappa"], cm_xgb$overall["Kappa"]),
Accuracy = c(cm_glmnet$overall["Accuracy"], cm_rf$overall["Accuracy"],
cm_svm$overall["Accuracy"], cm_xgb$overall["Accuracy"])
)
print(results)

In [ ]:
# Combined ROC Plot
plot(roc_glmnet, col = "purple", lwd = 2, main = "ROC Curves - All Models")
plot(roc_rf, col = "steelblue", lwd = 2, add = TRUE)
plot(roc_svm, col = "darkgreen", lwd = 2, add = TRUE)
plot(roc_xgb, col = "firebrick", lwd = 2, add = TRUE)
abline(a = 0, b = 1, lty = 2, col = "gray")
legend("bottomright", legend = c("GLMnet", "RF", "SVM", "XGB"),
col = c("purple", "steelblue", "darkgreen", "firebrick"),
lwd = 2)

In [ ]:
# Wrap tuned models into a resamples object
resamps <- resamples(list(
GLMnet = mod_glmnet,
RF = mod_rf,
SVM = mod_svm,
XGB = mod_xgb
))
# ROC Dotplot
dotplot(resamps, metric = "ROC", main = "ROC Dotplot - Tuned Models")

In [ ]:
# ROC Boxplot
bwplot(resamps, metric = "ROC", main = "ROC Boxplot - Tuned Models")

**Best Mode: XGBoost**

In [ ]:
library(pROC)
library(caret)
library(vip)

# Prepare test data
X_test <- test_clean %>% select(-treatment_outcome)
y_test <- test_clean$treatment_outcome

# Predict probabilities and classes
xgb_probs <- predict(mod_xgb, newdata = X_test, type = "prob")
xgb_preds <- predict(mod_xgb, newdata = X_test)

# Confusion Matrix
cm_xgb <- confusionMatrix(xgb_preds, y_test, positive = "Responder")
print(cm_xgb)

In [ ]:
# ROC Curve
roc_xgb <- roc(response = y_test,
predictor = xgb_probs$Responder,
levels = c("NonResponder", "Responder"),
direction = "<")
plot(roc_xgb, col = "red", main = "ROC Curve - XGBoost")

In [ ]:
auc_xgb <- auc(roc_xgb)
print(paste("AUC:", round(auc_xgb, 3)))

In [ ]:
# Confidence Interval
ci_xgb <- ci.auc(roc_xgb)
print(ci_xgb)

In [ ]:
# Variable Importance
vip(mod_xgb$finalModel)

In [ ]:
library(dplyr)

# Calculate proportion of responders per trial arm
prop_table <- df %>%
dplyr::filter(trial_arm %in% c("Drug 1", "Drug 3")) %>%
group_by(trial_arm, treatment_outcome) %>%
summarise(count = n(), .groups = "drop") %>%
group_by(trial_arm) %>%
mutate(total = sum(count),
                   proportion = count / total) %>%
dplyr::filter(treatment_outcome == "Responder")
# View proportions for Drug 1 and Drug 3
print(prop_table)

In [ ]:
# Calculate absolute and relative improvement
drug1_prop <- prop_table %>% dplyr::filter(trial_arm == "Drug 1") %>% pull(proportion)
drug3_prop <- prop_table %>% dplyr::filter(trial_arm == "Drug 3") %>% pull(proportion)
absolute_improvement <- drug3_prop - drug1_prop
relative_improvement <- (drug3_prop - drug1_prop) / drug1_prop * 100
cat("Absolute improvement in responder rate (Drug 3 vs Drug 1):", round(absolute_improvement, 3), "\n")

In [ ]:
cat("Relative improvement (% increase):", round(relative_improvement, 1), "%\n")

In [ ]:
# Calculate proportion with side effects per trial arm
side_effect_table <- df %>%
dplyr::filter(trial_arm %in% c("Drug 1", "Drug 3")) %>%
group_by(trial_arm, side_effect) %>%
summarise(count = n(), .groups = "drop") %>%
group_by(trial_arm) %>%
mutate(total = sum(count),
proportion = count / total) %>%
dplyr::filter(side_effect == "Yes")

# View proportions for Drug 1 and Drug 3
print(side_effect_table)

In [ ]:
# Calculate absolute and relative difference
drug1_se_prop <- side_effect_table %>% dplyr::filter(trial_arm == "Drug 1") %>% pull(proportion)
drug3_se_prop <- side_effect_table %>% dplyr::filter(trial_arm == "Drug 3") %>% pull(proportion)
absolute_diff_se <- drug3_se_prop - drug1_se_prop
relative_diff_se <- (drug3_se_prop - drug1_se_prop) / drug1_se_prop * 100
cat("Absolute difference in side effect rate (Drug 3 vs Drug 1):", round(absolute_diff_se, 3), "\n")

In [ ]:
cat("Relative difference (% change):", round(relative_diff_se, 1), "%\n")